In [2]:
from Fundamentals import Bayesian_Optimization
import numpy as np
import pymc as pm

In [2]:
def gaussian_model(lam, n_samples):
    
    gaussian_model = pm.Model()
        
    with gaussian_model:

        theta = pm.Normal("theta", mu=lam["mu_1"], sigma=lam["sigma_1"])
            
        Y_obs = pm.Normal("Y_obs", mu=theta, sigma=lam["sigma"], observed=1.)
        
    with gaussian_model:
        
        idata = pm.sample_prior_predictive(samples = n_samples)
        
    return idata


partitions = np.array([[-1000, -5], [-5, -1], [-1, 3], [3, 1000]])

expert_probs = [np.array([0.1, 0.2, 0.6, 0.1])]



BO = Bayesian_Optimization(pymc_sampling_func=gaussian_model,
                              J = 1,
                              alpha=None,
                              target_type="continuous",
                              target_samples=1000)


   

param_names = ["mu_1", "sigma_1", "sigma", "alpha"]
param_bounds = [[-10., 10.], [0.00001, 4.], [0.00001, 4.], [0.1, 20.]]
param_expected_vals = [-1, 3, 0.5, None]
param_weights = [1, 1, 1]

   
    
best_params = BO.optimize_hyperparams(param_names=param_names,
                                      param_bounds=param_bounds,
                                      param_expected_vals = param_expected_vals,
                                      param_weights = param_weights,
                                      partitions=partitions,
                                      expert_probs=expert_probs,
                                      n_trials=75)  


[INFO 06-15 12:25:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter mu_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 12:25:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 12:25:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 12:25:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter alpha. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter d

In [3]:
best_params

{'mu_1': -0.6267276708243035,
 'sigma_1': 3.456495458200379,
 'sigma': 0.4645866276015849,
 'alpha': 11.513152262972097}

In [4]:
BO.eval_function(best_params, partitions, expert_probs)

Sampling: [Y_obs, theta]


Array(15.47217, dtype=float32)

In [5]:
BO.get_model_probs(best_params, partitions)

Sampling: [Y_obs, theta]


[array([0.101, 0.351, 0.386, 0.162])]

In [16]:
def classification_model(lam, n_samples):
    
    RANDOM_SEED = 8927
    
    covs = np.array([[1.3, 0.7, 0.5, -0.7, -0.5],
                     [1, 0.5, 0.4, -0.8, 0],
                     [0.3, 0.7, 2, -2, 0.2]])
    
    classification_model = pm.Model()

    with classification_model:

        theta_1 = pm.Normal("theta_1", mu=lam["mu_1"], sigma=lam["sigma_1"])
        theta_2 = pm.Normal("theta_2", mu=lam["mu_2"], sigma=lam["sigma_2"])
        theta_3 = pm.Normal("theta_3", mu=lam["mu_3"], sigma=lam["sigma_3"])
        theta_4 = pm.Normal("theta_4", mu=lam["mu_4"], sigma=lam["sigma_4"])
        theta_5 = pm.Normal("theta_5", mu=lam["mu_5"], sigma=lam["sigma_5"])
        
        theta = pm.math.stack([theta_1, theta_2, theta_3, theta_4, theta_5])
        
        p = pm.math.dot(covs, theta)

        p = 0.5 + 0.5 * pm.math.erf(p / pm.math.sqrt(2))
                    
        Y_obs = pm.Bernoulli("Y_obs", p = p, observed=np.ones(covs.shape[0]))
        
    with classification_model:
        idata = pm.sample_prior_predictive(random_seed=RANDOM_SEED, samples = n_samples)
        
    return idata



BO = Bayesian_Optimization(pymc_sampling_func=classification_model,
                              J = 3,
                              alpha=None,
                              target_type="discrete",
                              target_samples=1000)
   

param_names = ["mu_1", "sigma_1",
               "mu_2", "sigma_2",
               "mu_3", "sigma_3",
               "mu_4", "sigma_4",
               "mu_5", "sigma_5",
               "alpha"]

partitions = np.array([0, 1])

expert_probs = [np.array([0.65, 0.35]),
                np.array([0.3, 0.7]),
                np.array([0.2, 0.8])]



param_bounds = [[-6., 6.], [0.00001, 2.],
                [-6., 6.], [0.00001, 2.],
                [-6., 6.], [0.00001, 2.],
                [-6., 6.], [0.00001, 2.],
                [-6., 6.], [0.00001, 2.],
                [0.1, 200.]]

param_expected_vals = [1, 1,
                       -0.2, 1,
                       -1, 1,
                       1, 1,
                       -0.5, 1,
                       None]



param_weights = [1, 0.5,
                 1, 0.5,
                 1, 0.5,
                 1, 0.5,
                 1, 0.5]
   
    
best_params = BO.optimize_hyperparams(param_names=param_names,
                                      param_bounds=param_bounds,
                                      param_expected_vals = param_expected_vals,
                                      param_weights = param_weights,
                                      partitions=partitions,
                                      expert_probs=expert_probs,
                                      n_trials=100)  



[INFO 06-17 09:35:14] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter mu_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 09:35:14] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 09:35:14] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter mu_2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 09:35:14] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma_2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter 

In [17]:
best_params

{'mu_1': -0.6742818697838411,
 'sigma_1': 1.2223770539815504,
 'mu_2': 0.42187642827987304,
 'sigma_2': 0.7019586844353061,
 'mu_3': -0.07376802987290532,
 'sigma_3': 1.3640267897356686,
 'mu_4': -1.287651261486726,
 'sigma_4': 0.7791307494208056,
 'mu_5': 2.1435902665479816,
 'sigma_5': 0.5471019654570807,
 'alpha': 28.287572641381214}

In [18]:
BO.eval_function(best_params, partitions, expert_probs)

Sampling: [Y_obs, theta_1, theta_2, theta_3, theta_4, theta_5]


Array(88.81602, dtype=float32)

In [19]:
BO.get_model_probs(best_params, partitions)

Sampling: [Y_obs, theta_1, theta_2, theta_3, theta_4, theta_5]


[array([0.644, 0.356]), array([0.386, 0.614]), array([0.199, 0.801])]

In [33]:
partitions = np.array([0, 1])  ## We have a classification task


## input for the expert probabilities (J=6 covariate sets in total)

expert_probs = [np.array([0.8854, 0.1146]),
                np.array([0.7857, 0.2143]),
                np.array([0.2237, 0.7763]),
                np.array([0.9023, 0.0977]),
                np.array([0.8167, 0.1833]),
                np.array([0.7857, 0.2143])]


## the logistic regression model

def trauma_center_model(lam, n_samples):
    
    
    ## values for the covariates. The first row (column when transposed) is for the intercept
    
    covs = np.array([[1,1,1,1,1,1],
                     [25,25,41,41,33,33],
                     [7.84, 3.34, 3.34, 7.84, 5.74, 5.74],
                     [60, 10, 60, 10, 35, 35],
                     [0,0,1,1,0,1],
                     [0,0,60,10,0,35]]).T
    
    trauma_center_model = pm.Model()

    with trauma_center_model:

        theta_1 = pm.Normal("theta_1", mu=lam["mu_1"], sigma=lam["sigma_1"])
        theta_2 = pm.Normal("theta_2", mu=lam["mu_2"], sigma=lam["sigma_2"])
        theta_3 = pm.Normal("theta_3", mu=lam["mu_3"], sigma=lam["sigma_3"])
        theta_4 = pm.Normal("theta_4", mu=lam["mu_4"], sigma=lam["sigma_4"])
        theta_5 = pm.Normal("theta_5", mu=lam["mu_5"], sigma=lam["sigma_5"])
        theta_6 = pm.Normal("theta_6", mu=lam["mu_6"], sigma=lam["sigma_6"])
        
        theta = pm.math.stack([theta_1, theta_2, theta_3, theta_4, theta_5, theta_6])
        
        xTtheta = pm.math.dot(covs, theta)
        
        p = pm.math.exp(xTtheta) / (1 + pm.math.exp(xTtheta))
        
        Y_obs = pm.Bernoulli("Y_obs", p = p, observed=np.ones(covs.shape[0]))
        
    with trauma_center_model:
        idata = pm.sample_prior_predictive(samples=n_samples)
        
    return idata



BO = Bayesian_Optimization(pymc_sampling_func=trauma_center_model,
                              J = 6,
                              alpha=None,
                              target_type="discrete",
                              target_samples=2000)
   
   

   

param_names = ["mu_1", "sigma_1",
               "mu_2", "sigma_2",
               "mu_3", "sigma_3",
               "mu_4", "sigma_4",
               "mu_5", "sigma_5",
               "mu_6", "sigma_6",
               "alpha"]



## the bounds for each hyperparameter

param_bounds = [[-4., 4.], [0.00001, 3.],
                [-4., 4.], [0.00001, 3.],
                [-4., 4.], [0.00001, 3.],
                [-4., 4.], [0.00001, 3.],
                [-4., 4.], [0.00001, 3.],
                [-4., 4.], [0.00001, 3.],
                [0.1, 100.]]


## expected values (taken from the thesis, these correspond to the posterior means and standard deviations from the original study)

param_expected_vals = [-1.8, 1.1,
                       0.07, 0.02,
                       -0.6, 0.14,
                       0.05, 0.01,
                       -1.1, 1.06,
                       -0.02, 0.03,
                       None]


## Full confidence for each expected hyperparameter value (all weights=1)

param_weights = np.ones(len(param_bounds))
   
   
   
best_params = BO.optimize_hyperparams(param_names=param_names,
                                      param_bounds=param_bounds,
                                      param_expected_vals = param_expected_vals,
                                      param_weights = param_weights,
                                      partitions=partitions,
                                      expert_probs=expert_probs,
                                      n_trials=100)  

[INFO 06-17 10:01:07] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter mu_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 10:01:07] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma_1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 10:01:07] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter mu_2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-17 10:01:07] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter sigma_2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter 

In [38]:
best_params

{'mu_1': -0.8850895542373252,
 'sigma_1': 0.10227424350928956,
 'mu_2': -2.037586665993198,
 'sigma_2': 2.5852760459612303,
 'mu_3': -0.25233399425053626,
 'sigma_3': 0.6134568808942173,
 'mu_4': -0.9610456346839338,
 'sigma_4': 3.0,
 'mu_5': -0.3227410221308653,
 'sigma_5': 1e-05,
 'mu_6': 2.037987209002746,
 'sigma_6': 0.9668938262956811,
 'alpha': 8.39287445417239}

In [42]:
BO.get_model_probs(best_params, partitions)

Sampling: [Y_obs, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6]


[array([0.728, 0.272]),
 array([0.814, 0.186]),
 array([0.5545, 0.4455]),
 array([0.764, 0.236]),
 array([0.7875, 0.2125]),
 array([0.5975, 0.4025])]

In [43]:
expert_probs

[array([0.8854, 0.1146]),
 array([0.7857, 0.2143]),
 array([0.2237, 0.7763]),
 array([0.9023, 0.0977]),
 array([0.8167, 0.1833]),
 array([0.7857, 0.2143])]

In [50]:
BO.eval_function(best_params, partitions, expert_probs)

Sampling: [Y_obs, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6]


Array(5.922669, dtype=float32)